In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo

Be $A$ a matrix and $A = U^T \lambda U$, with $\lambda_{ij} = \lambda_i \delta_{ij}$ where $\lambda$ are the eigenvalues of $A$, then
$A^{\frac{1}{2}}= U^T \lambda^{\frac{1}{2}} U$ in the sense, that $$A^{\frac{1}{2}} A^{\frac{1}{2}} = A$$ since $$A^{\frac{1}{2}} A^{\frac{1}{2}} = U^T \lambda^{\frac{1}{2}} U U^T \lambda^{\frac{1}{2}} U = U^T \lambda U$$

In [2]:
def sqrt_matrix(in_mat):
    eigval, eigvec = np.linalg.eigh(in_mat)
    sqrt_eigval = np.sqrt(eigval)
    return(np.matmul(eigvec.T,np.matmul(np.diag(sqrt_eigval),eigvec)))


$$
\begin{split}
E_x & = \sum_i^N \sum_j^N \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M c_{i,\mu} c_{j,\nu} c_{i,\kappa} c_{j,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_x & = \sum_i^N \sum_j^N  \left [ i j | i j \right ] \\
E_x & = \sum_i^N \sum_j^N  \langle  i j | j i  \rangle 
\end{split}
$$

In [3]:
def exchange_energy(Fouridx, C, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,N):
        for j in range(0,N):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += C[mu,i]*C[nu,j]*C[kappa,i]*C[lamda,j]*Fouridx[mu,nu,kappa,lamda]

    return energy


$$
\begin{split}
E_H & = \sum_i^N \sum_j^N \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M c_{i,\mu} c_{j,\nu} c_{i,\kappa} c_{j,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_H & = \sum_i^N \sum_j^N  \left [ i i | j j \right ] \\
E_H & = \sum_i^N \sum_j^N  \langle  i j | i j \rangle 
\end{split}
$$

In [4]:
def hartree_energy(Fouridx, C, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,N):
        for j in range(0,N):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += C[mu,i]*C[nu,i]*C[kappa,j]*C[lamda,j]*Fouridx[mu,nu,kappa,lamda]

    return energy


In contrast to the previous expression for the hartree energy, this expression runs over all orbitals {$\phi_a$}, which are the natural orbitals.  $n_a$ are the occupation numbers. $c_{a, \mu}$ are the coefficients of the matrix that diagonalizes $P$, which is the basis set representation of $\gamma^{(1)}$.

$$
\begin{split}
E_H[\gamma_1] & = \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  c_{a,\mu} c_{b,\nu} c_{a,\kappa} c_{b,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_H[\gamma_1] & = \iint \sum_a^M \sum_b^M n_a n_b \frac{\phi_a(r)^{\ast} \phi_a(r) \phi_b(r')^{\ast}  \phi_b(r')}{|r-r'|} dr dr' \\
E_H[\gamma_1] & = \iint \frac{\gamma_1(r,r) \gamma_1(r',r')}{|r-r'|} dr dr' \\
E_H[\gamma_1] & = \sum_a^M \sum_b^M n_a n_b \left [ a a  | b b \right ] \\
E_H[\gamma_1] & = \sum_a^N \sum_b^N n_a n_b \langle  a b | a b \rangle 
\end{split}
$$

In [5]:
def ONERDMFT_hartree_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,M):
        for j in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[i]*n[j]*C[mu,i]*C[nu,i]*C[kappa,j]*C[lamda,j]*Fouridx[mu,nu,kappa,lamda]

    return energy



$$
\begin{split}
E_x[\gamma_1] & = \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  c_{a,\mu} c_{b,\nu} c_{b,\kappa} c_{a,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_x[\gamma_1] & = \iint \sum_a^M \sum_b^M n_a n_b \frac{\phi_a(r)^{\ast} \phi_b(r) \phi_b(r')^{\ast}  \phi_a(r')}{|r-r'|} dr dr' \\
E_x[\gamma_1] & = \iint \frac{\gamma_1(r,r') \gamma_1(r',r)}{|r-r'|} dr dr' \\
\end{split}
$$

In [6]:
def ONERDMFT_exchange_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,M):
        for j in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[i]*n[j]*C[mu,i]*C[nu,j]*C[kappa,j]*C[lamda,i]*Fouridx[mu,nu,kappa,lamda]

    return energy

$$
\begin{split}
E_{xc}[\gamma_1] & = \sum_a^M \sum_b^M \sqrt{n_a n_b} \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  c_{a,\mu} c_{b,\nu} c_{b,\kappa} c_{a,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_{xc}[\gamma_1] & = \iint \sum_a^M \sum_b^M \sqrt{n_a n_b} \frac{\phi_a(r)^{\ast} \phi_b(r) \phi_b(r')^{\ast}  \phi_a(r')}{|r-r'|} dr dr' \\
E_{xc}[\gamma_1] & = \iint \frac{\gamma_1^{\frac{1}{2}}(r,r') \gamma_1^{\frac{1}{2}}(r',r)}{|r-r'|} dr dr' \\
\end{split}
$$

In [7]:
def ONERDMFT_Mueller_functional(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,M):
        for j in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[i]*n[j])*C[mu,i]*C[nu,j]*C[kappa,j]*C[lamda,i]*Fouridx[mu,nu,kappa,lamda]

    return energy

In [8]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 in the case of  case 16 distinct elements
eri = mol.intor('int2e')
print(f"The System {mol.atom} has {eri.size} or {eri.shape[0]}^4 elements in 2-electron-intergrals/4-index-integrals matrix with the {mol.basis}-basis")



The System 
    He    0.    0.    0.
 has 16 or 2^4 elements in 2-electron-intergrals/4-index-integrals matrix with the 6-31g-basis


In [9]:
print("Overlap Integrals")
S = mol.intor('int1e_ovlp')
for i in range(0,S.shape[0]):
    for j in range(0,S.shape[1]):
        print(f"S_{i}{j} = {S[i,j]}" )
SMH = sqrt_matrix(S)

Overlap Integrals
S_00 = 1.0
S_01 = 0.6341477386718484
S_10 = 0.6341477386718484
S_11 = 1.0


In [10]:
## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

print("*"*24)
print("MO-Coefficent matrix")
for mu,AO in enumerate(mf.mo_coeff):
    print(f"Coefficients of mu={mu} {AO}")
print("*"*24)

#print("den ?")
#print(np.matmul(mf.mo_coeff.T,mf.mo_coeff))
#print("*"*24)

converged SCF energy = -2.85516042615445
************************
MO-Coefficent matrix
Coefficients of mu=0 [ 0.59208126 -1.14981805]
Coefficients of mu=1 [0.51358601 1.1869588 ]
************************


Energy Components from PySCF Tools:
$$ T + V_{eK} = Tr[h \gamma^{(1)}];  U = \frac{1}{2} Tr[J \gamma^{(1)}]; E_x = -\frac{1}{4} Tr[K \gamma^{(1)}]$$


In [11]:
# get j, k and gamma (1RDM) matrix from hf, 
J = mf.get_j()
K = mf.get_k()
h = mf.get_hcore()
C = mf.mo_coeff
gamma = mf.make_rdm1()

# calculate the energy components to see what they are from the matrices the mf object offers you 
print("Energy Components from PySCF Tools:")
print(f"h_0 = {np.trace(np.matmul(h,gamma))}; U = {1/2*np.trace(np.matmul(J,gamma))}; E_x =  {1/4.*np.trace(np.matmul(K,gamma))}")
print(f"h_0 + U + E_x = {np.trace(np.matmul(h,gamma))+1/2.*np.trace(np.matmul(J,gamma))-1/4.*np.trace(np.matmul(K,gamma))}")

Energy Components from PySCF Tools:
h_0 = -3.882067595029823; U = 2.0538143377507527; E_x =  1.0269071688753764
h_0 + U + E_x = -2.8551604261544465


Energy Components 4IDX:

$$U = 2 E_h[C]; E_x = -E_x[C]$$

In [12]:
# this should also work
print("Energy Components from direct calculations:")
print(f"U = {2*hartree_energy(eri, mf.mo_coeff, mol)} E_x = {exchange_energy(eri, mf.mo_coeff, mol)} ")

Energy Components from direct calculations:
U = 2.0538143377507527 E_x = 1.0269071688753764 


In [13]:
print("Text-Book Gamma")
N = mol.nelec[0]
print(N)
MPgamma=np.matmul(C[:,0:N],C[:,0:N].T)*2
for mu,AO in enumerate(MPgamma):
    print(f"Coefficients of mu={mu} {AO}")

print("PySCF Gamma")
for mu,AO in enumerate(gamma):
    print(f"Coefficients of mu={mu} {AO}")




Text-Book Gamma
1
Coefficients of mu=0 [0.70112044 0.60816931]
Coefficients of mu=1 [0.60816931 0.52754118]
PySCF Gamma
Coefficients of mu=0 [0.70112044 0.60816931]
Coefficients of mu=1 [0.60816931 0.52754118]


In [14]:
# this serves to show  that, not C is orthogonal, but S^{-1/2} C
print(np.matmul(S,np.matmul(C,C.T)))

[[1.00000000e+00 6.39451579e-17]
 [0.00000000e+00 1.00000000e+00]]


In [15]:
occu, naturalC = np.linalg.eigh(gamma)
print(f"should be natural occupation numbers, the sum of occupation numbers, i.e. N is {np.sum(occu)}")
print(occu)
print("*"*24)
print("Natural Orbital LC-Coefficent matrix")
for mu,AO in enumerate(naturalC):
    print(f"Coefficients of mu={mu} {AO}")
print("*"*24)
print(np.matmul(naturalC, naturalC.T))

should be natural occupation numbers, the sum of occupation numbers, i.e. N is 1.2286616201239735
[5.55111512e-17 1.22866162e+00]
************************
Natural Orbital LC-Coefficent matrix
Coefficients of mu=0 [ 0.65525756 -0.75540554]
Coefficients of mu=1 [-0.75540554 -0.65525756]
************************
[[1.00000000e+00 1.92290416e-17]
 [1.92290416e-17 1.00000000e+00]]


In [16]:
# this is always off by a factor of 4 but it works, i.e. I get reasonable numbers
#which means here I deal as before only with quantities that are given 
# with respect to a basis set, never the quanities itselff.

print("Hartree DM, 4I", 1/2*ONERDMFT_hartree_energy(eri, naturalC, occu, mol), 2*hartree_energy(eri, mf.mo_coeff, mol))
print("Exchange DM, 4I", 1/4*ONERDMFT_exchange_energy(eri, naturalC, occu, mol), exchange_energy(eri, mf.mo_coeff, mol))

Hartree DM, 4I 2.0538143377507523 2.0538143377507527
Exchange DM, 4I 1.0269071688753761 1.0269071688753764


In [17]:
J_PY = 1/2*np.trace(np.matmul(J,gamma))
J_4I = 2*hartree_energy(eri, mf.mo_coeff, mol)
J_DM = 1/2*ONERDMFT_hartree_energy(eri, naturalC, occu, mol)

print(J_PY, J_4I, J_DM)

2.0538143377507527 2.0538143377507527 2.0538143377507523


In [18]:
for i, n  in enumerate(occu):
    if abs(n) < 1e-10:
        occu[i] = 0 

print("Mueller Correlation Energy:", 1/4*ONERDMFT_Mueller_functional(eri, naturalC, occu, mol)-1/4*ONERDMFT_exchange_energy(eri, naturalC, occu, mol))

Mueller Correlation Energy: -0.19111385356716337
